In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import collections

import numpy as np
import pandas as pd

import evaluate
from transformers import AutoModelForQuestionAnswering
from transformers import Trainer
from transformers import TrainingArguments
from tqdm.auto import tqdm
import torch

ModuleNotFoundError: ignored

In [ ]:
!nvidia-smi

torch.cuda.is_available()

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



False

# Set Experiment Parameters

In [ ]:
#set model to use

#expt set up: model/checkpoint id (int), use_full_context (True/False), merge (True/False)
expts=[
    '''
    to be subsequently uncommented out
    [0, True, False], #bert, full context, not merged
    [0, False, False], #bert, shortened context, not merged
    [0, False, True], #bert, full context, merged
    [1, True, False], #roberta, full context, not merged
    [1, False, False], #roberta, shortened context, not merged
    [1, False, True], #roberta, full context, merged
    '''
    [2, True, False], #scibert, full context, not merged
    [3, False, False], #gpt2, shortened context, not merged
    [3, True, False], #gpt2, full context, not merged
    [3, False, True], #gpt2, shortened context, merged
    [3, True, True], #gpt2, full context, merged
    [4, False, False], #t5, shortened context, not merged
    [4, True, False], #t5, full context, not merged
    [4, False, True], #t5, shortened context, merged
    [4, True, True] #t5, full context, merged
       ]
models = ["bert-base-cased", "roberta-base", "allenai/scibert_scivocab_uncased", "gpt2", "t5-base"]
checkpoint_names = ["bert-finetuned-squad", "roberta-finetuned-squad", "allenai/scibert-finetuned-squad", "gpt2-finetuned-squad", "t5-finetuned-squad"]

#i = 1
expt_id = 0
curr_expt = expts[expt_id] 
model_id = curr_expt[0]

#set dataset to use
use_full_context = curr_expt[1]
merge = curr_expt[2]

print('model name:' + models[model_id] + ', checkpoint name: ' + checkpoint_names[model_id] + ',use full context: ' + str(use_full_context) + ', merged dataset: ' + str(merge))

model name:allenai/scibert_scivocab_uncased, checkpoint name: allenai/scibert-finetuned-squad,use full context: True, merged dataset: False


# Load CovidQA Dataset

In [ ]:
# NON-SHORTENED CONTEXT VERSION
df = pd.read_json('https://raw.githubusercontent.com/deepset-ai/COVID-QA/master/data/question-answering/COVID-QA.json')

qa_list = []
for index, row in df.iterrows():
  for i in range(len(row['data']['paragraphs'][0]['qas'])):
    qa_temp_list = []
    qa_temp_list.append(row['data']['paragraphs'][0]['context'])
    qa_temp_list.append(row['data']['paragraphs'][0]['qas'][i]['question'])
    qa_temp_list.append(row['data']['paragraphs'][0]['qas'][i]['id'])
    answer = {}
    answer['text'] = []
    answer['answer_start'] = []
    for j in range(len(row['data']['paragraphs'][0]['qas'][i]['answers'])):
      answer['text'].append(row['data']['paragraphs'][0]['qas'][i]['answers'][j]['text'])
      answer['answer_start'].append(row['data']['paragraphs'][0]['qas'][i]['answers'][j]['answer_start'])
    qa_temp_list.append(answer)
    qa_list.append(qa_temp_list)
long_qa_df = pd.DataFrame(qa_list, columns = ['context', 'question', 'id', 'answers'])
long_qa_df

,context,question,id,answers
0,Functional Genetic Variants in DC-SIGNR Are As...,What is the main cause of HIV-1 infection in c...,262,{'text': ['Mother-to-child transmission (MTCT)...
1,Functional Genetic Variants in DC-SIGNR Are As...,What plays the crucial role in the Mother to C...,276,{'text': ['DC-SIGNR plays a crucial role in MT...
2,Functional Genetic Variants in DC-SIGNR Are As...,How many children were infected by HIV-1 in 20...,278,"{'text': ['more than 400,000 children were inf..."
3,Functional Genetic Variants in DC-SIGNR Are As...,What is the role of C-C Motif Chemokine Ligand...,316,"{'text': ['High copy numbers of CCL3L1, a pote..."
4,Functional Genetic Variants in DC-SIGNR Are As...,What is DC-GENR and where is it expressed?,305,{'text': ['Dendritic cell-specific ICAM-grabbi...
...,...,...,...,...
2014,"Ebola Virus Maintenance: If Not (Only) Bats, W...",What is the structure of the Ebolavirus?,5315,"{'text': ['single-strand RNA filoviruses'], 'a..."
2015,"Ebola Virus Maintenance: If Not (Only) Bats, W...",When was the West African Ebolavirus outbreak?,5316,"{'text': ['2013-2016'], 'answer_start': [2546]}"
2016,"Ebola Virus Maintenance: If Not (Only) Bats, W...",What animals are considered to be maintenance ...,5317,"{'text': ['African bats'], 'answer_start': [40..."
2017,"Ebola Virus Maintenance: If Not (Only) Bats, W...",What do circles indicate in Figure 1?,5318,{'text': ['a maintenance function play by the ...


## Shortened Context Version

In [3]:
# SHORTENED CONTEXT VERSION
df = pd.read_json('https://raw.githubusercontent.com/deepset-ai/COVID-QA/master/data/question-answering/COVID-QA.json')

qa_list = []
for index, row in df.iterrows():
  paragraph_list = row['data']['paragraphs'][0]['context'].split('\n\n')
  for i in range(len(row['data']['paragraphs'][0]['qas'])):
    qa_temp_list = []
    short_context = '\n\n'.join(list(filter(lambda x : row['data']['paragraphs'][0]['qas'][i]['answers'][0]['text'] in x, paragraph_list)))
    qa_temp_list.append(short_context)
    qa_temp_list.append(row['data']['paragraphs'][0]['qas'][i]['question'])
    qa_temp_list.append(row['data']['paragraphs'][0]['qas'][i]['id'])

    ans_text = row['data']['paragraphs'][0]['qas'][i]['answers'][0]['text']
    ans_start = row['data']['paragraphs'][0]['qas'][i]['answers'][0]['answer_start']
    if (len(short_context)) > 0:
      ans_start = short_context.index(str(ans_text))
    answer = {'text': [ans_text],
              'answer_start': [ans_start]}
    qa_temp_list.append(answer)
    qa_list.append(qa_temp_list)
new_qa_df = pd.DataFrame(qa_list, columns = ['context', 'question', 'id', 'answers'])
new_qa_df = new_qa_df.drop(new_qa_df['context'].loc[new_qa_df['context'].str.len() <= 0].index)
new_qa_df.head()

,context,question,id,answers
0,Abstract: BACKGROUND: Mother-to-child transmis...,What is the main cause of HIV-1 infection in c...,262,{'text': ['Mother-to-child transmission (MTCT)...
1,Abstract: BACKGROUND: Mother-to-child transmis...,What plays the crucial role in the Mother to C...,276,{'text': ['DC-SIGNR plays a crucial role in MT...
2,"Text: Without specific interventions, the rate...",How many children were infected by HIV-1 in 20...,278,"{'text': ['more than 400,000 children were inf..."
3,"Beside DC-SIGNR, other HIV-1 receptors are kno...",What is the role of C-C Motif Chemokine Ligand...,316,"{'text': ['High copy numbers of CCL3L1, a pote..."
4,Dendritic cell-specific ICAM-grabbing non-inte...,What is DC-GENR and where is it expressed?,305,{'text': ['Dendritic cell-specific ICAM-grabbi...


## Select whether to use full context or shortened context

In [ ]:
if use_full_context:
  new_qa_df = long_qa_df

In [ ]:
new_qa_df.shape

(2019, 4)

In [ ]:
from datasets import Dataset

qa_dataset = Dataset.from_pandas(new_qa_df)
qa_dataset = qa_dataset.train_test_split(train_size = 0.7, seed = 42)
split_qa_dataset = qa_dataset['test'].train_test_split(train_size = 0.8, seed = 42)
split_qa_dataset['validation'] = split_qa_dataset.pop('test')
split_qa_dataset['test'] = split_qa_dataset.pop('train')
split_qa_dataset['train'] = qa_dataset['train']

qa_dataset = split_qa_dataset

In [ ]:
qa_dataset

DatasetDict({
    validation: Dataset({
        features: ['context', 'question', 'id', 'answers'],
        num_rows: 122
    })
    test: Dataset({
        features: ['context', 'question', 'id', 'answers'],
        num_rows: 484
    })
    train: Dataset({
        features: ['context', 'question', 'id', 'answers'],
        num_rows: 1413
    })
})

In [ ]:
print('Id: ', qa_dataset['train'][0]['id'])
print('Context: ', qa_dataset['train'][0]['context'])
print('Question: ', qa_dataset['train'][0]['question'])
print('Answer: ', qa_dataset['train'][0]['answers'])

Id:  2512
Context:  Chikungunya: A Potentially Emerging Epidemic?

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2860491/

SHA: f7c3160bef4169d29e2a8bdd79dd6e9056d4774c

Authors: Thiboutot, Michelle M.; Kannan, Senthil; Kawalekar, Omkar U.; Shedlock, Devon J.; Khan, Amir S.; Sarangan, Gopalsamy; Srikanth, Padma; Weiner, David B.; Muthumani, Karuppiah
Date: 2010-04-27
DOI: 10.1371/journal.pntd.0000623
License: cc-by

Abstract: Chikungunya virus is a mosquito-borne emerging pathogen that has a major health impact in humans and causes fever disease, headache, rash, nausea, vomiting, myalgia, and arthralgia. Indigenous to tropical Africa, recent large outbreaks have been reported in parts of South East Asia and several of its neighboring islands in 2005–07 and in Europe in 2007. Furthermore, positive cases have been confirmed in the United States in travelers returning from known outbreak areas. Currently, there is no vaccine or antiviral treatment. With the threat of an emerging global pan

# Merge SQuAD and CovidQA Datasets
* Upsample CovidQA to be same size as SQuAD

In [ ]:
from datasets import load_dataset
from sklearn.utils import resample

def label_dataset(input, label):
    return {"type": label}

if merge:
  squad_dataset = load_dataset("squad")
  squad_train_df = pd.DataFrame(squad_dataset['train'])
  # squad_train_df = squad_dataset["train"][:]
  qa_train_df = pd.DataFrame(qa_dataset['train'])
  squad_train_df['id'] = squad_train_df['id'].astype(str)
  qa_train_df['id'] = qa_train_df['id'].astype(str)

  squad_validation_df = pd.DataFrame(squad_dataset["validation"])
  qa_validation_df = pd.DataFrame(qa_dataset['validation'])
  squad_validation_df['id'] = squad_validation_df['id'].astype(str)
  qa_validation_df['id'] = qa_validation_df['id'].astype(str)

  # # Merge train
  # # Upsample
  qa_train_upsample = resample(qa_train_df,
              replace=True,
              n_samples=len(squad_train_df),
              random_state=42)
  merged_train = pd.concat([squad_train_df,qa_train_upsample],ignore_index=True)
  merged_train_dataset =  Dataset.from_pandas(merged_train)
  # # Merge validation
  # # Upsample
  qa_validation_upsample = resample(qa_validation_df,
              replace=True,
              n_samples=len(squad_validation_df),
              random_state=42)

  merged_validation = pd.concat([squad_validation_df,qa_validation_upsample],ignore_index=True)
  merged_validation_dataset = Dataset.from_pandas(merged_validation)
else:
  merged_train_dataset = qa_dataset['train']
  merged_validation_dataset = qa_dataset['validation']

In [ ]:
print(merged_train_dataset.shape)
print(merged_validation_dataset.shape)

(1413, 4)
(122, 4)


# Import Pre-Trained Models

In [ ]:
from transformers import AutoTokenizer


In [ ]:
model_name = models[model_id]
model_checkpoint = checkpoint_names[model_id]
print(model_name)
print(model_checkpoint)

allenai/scibert_scivocab_uncased
allenai/scibert-finetuned-squad


In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("roberta-base")
tokenizer = AutoTokenizer.from_pretrained(model_name)

OSError: ignored

In [ ]:
|max_length = 384
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length = max_length,
        truncation = "only_second",
        stride = stride,
        return_overflowing_tokens = True,
        return_offsets_mapping = True,
        padding = "max_length",
    )

    # Each member of offset_mapping is list of (start, stop) tuples representing span in context that tokens correspond to
    # Sample map is list of indices of samples that each feature is mapped to
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    
    # Iterate through features
    # Multiple features can be mapped to one sample due to truncation
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        # sequence_ids is list of 1s and 0s - 0 for question token and 1 for context token
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
if merge:
  train_dataset = merged_train_dataset.map(
      preprocess_training_examples,
      batched = True,
      remove_columns = merged_train_dataset.column_names
  )
else: 
  train_dataset = qa_dataset['train'].map(
      preprocess_training_examples,
      batched = True,
      remove_columns = qa_dataset['train'].column_names
      )

NameError: ignored

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length = max_length,
        truncation = "only_second",
        stride = stride,
        return_overflowing_tokens = True,
        return_offsets_mapping = True,
        padding = "max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
if merge:
  validation_dataset = merged_validation_dataset.map(
      preprocess_validation_examples,
      batched = True,
      remove_columns = merged_validation_dataset.column_names,
  )
else:
  validation_dataset = qa_dataset["validation"].map(    
      preprocess_validation_examples,
      batched = True,
      remove_columns = qa_dataset["validation"].column_names,
      )

  0%|          | 0/1 [00:00<?, ?ba/s]

NameError: ignored

In [ ]:
metric = evaluate.load("squad")

n_best = 20
max_answer_length = 200

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    for x in range(len(predicted_answers)):
      predicted_answers[x]['id'] = str(predicted_answers[x]['id'])

    for y in range(len(theoretical_answers)):
      theoretical_answers[y]['id'] = str(theoretical_answers[y]['id'])
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
model_checkpoint

'allenai/scibert_scivocab_uncased'

In [ ]:
#model = AutoModelForQuestionAnswering.from_pretrained("roberta-base")
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized f

## Set Training Arguments

In [ ]:
args = TrainingArguments(
    model_checkpoint,
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)
#print(trainer)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 39666
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14877
  Number of trainable parameters = 109329410
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [ ]:
trainer.save_model('./content/saved_model')

# Results

## Bert Finetuned Squad results

In [ ]:
#using normal context
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, qa_dataset["validation"])

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****

  Num examples = 4154

  Batch size = 8

100%

122/122 [00:07<00:00, 16.24it/s]

{'exact_match': 16.39344262295082, 'f1': 41.12005635374793}

In [ ]:
#using shortened context
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, qa_dataset["validation"])

## Roberta Finetuned Squad results

In [ ]:
#using normal context
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, qa_dataset["validation"])

In [ ]:
#using shortened context
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, qa_dataset["validation"])

The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.

***** Running Prediction *****

  Num examples = 3923

  Batch size = 8

100%

122/122 [00:05<00:00, 19.96it/s]

{'exact_match': 23.770491803278688, 'f1': 52.758817529968965}

## Scibert results

In [ ]:
#using normal context
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, qa_dataset["validation"])

In [ ]:
#using merged shortened context
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, qa_dataset["validation"])